University of Michigan - ROB 201 Calculus for the Modern Engineer

---


# Julia HW07: Chapters 6.5-7.5 

### Topics Covered
- Path Planning
- Fundamental Theorems of Calculus
- Software Tools for Antiderivatives
  - SymPy
  - LLMs
  - Wolfram Alpha Pro

<br>

<p align="center">
  <img src="data/HW07KangarooRobotV02.png", width="30%">
</p>

> ChatGPT 4o was asked to place the Second Fundamental Theorem of Calculus on the robot kangaroo's chest...oh well!

# What is Path Planning?

Path planning is the process of figuring out how a robot can move from one place to another while avoiding obstacles along the way. Whether it’s a robot arm reaching for an object or a self-driving car navigating city streets, path planning plays a key role in making sure the robot moves efficiently, safely, and realistically. This involves more than just drawing a straight line from point A to point B — the robot’s movement has to respect physical constraints (like how it can turn or accelerate) and the layout of its environment. Depending on the application, there are many techniques to handle this, including grid-based methods, sampling-based algorithms like RRT (Rapidly-exploring Random Tree), and search algorithms such as A* and Dijkstra’s. Good path planning is what allows robots to operate on their own in unpredictable environments, whether they’re on a factory floor, navigating a warehouse, or driving on the road.

To learn more, check out this article: [arXiv:2108.06699](https://arxiv.org/abs/2108.06699)

<table>
  <tr>
    <td><img src="data/HW07map1.png"  width="80%"></td>
    <td><img src="data/HW07map3.png"  width="76%"></td>
  </tr>
</table>

### Optimality Criteria


In path planning for robotics, various optimality criteria can be used to determine the best path from a starting point to a destination. These criteria are often application-dependent and can significantly influence the efficiency and effectiveness of the robot’s navigation. Here are some of the common optimality criteria:

1. The simplest and most common goal is to minimize the distance traveled. Algorithms like Dijkstra’s and A* are great at finding the shortest route and are widely used in applications where speed matters (e.g. delivery robots or autonomous cars). Here, we illustrate the use of JuMP.

2. Sometimes the shortest path isn’t the fastest. If there are speed limits, traffic, or moving obstacles, travel time becomes a more important factor than distance.

3. For robots that run on batteries, conserving energy can be more important than taking the shortest or fastest route. Slower paths with easier terrain might actually be better in these cases.

4. In risky environments (like disaster zones or construction sites), safety becomes the top priority. This might mean keeping a wide margin from hazards or choosing more stable ground over a shorter or faster route.

5. In some applications, especially those involving delicate tasks like surgical robots or robotic arms in manufacturing, the robot’s movement needs to be smooth and precise. Here, we want to minimize sudden turns or jerky motions that could cause damage or error.


Path planning is all about getting from Point A to Point B, but how you get there depends on the robot’s priorities. A Mars rover, for instance, might care more about energy efficiency and safety than getting there quickly. A delivery drone might care more about speed.

In the work below, we’ll keep it simple: you’ll plan the **shortest path** from Point A to Point B while **avoiding obstacles**. We’ll use optimization tools like JuMP to help find that path.

<br>

## Example: Design a Path from Point A = [0.0, 0.0] to Point B = [10.0, 10.0], While Avoiding a Circular Obstacle

In [ ]:
using JuMP, Ipopt, Plots

# Define the start and end points, obstacle characteristics
start_point = [0.0, 0.0]
end_point = [10.0, 10.0]
circle_center = [4.0, 4.0]
circle_radius = 3.0

# Define a JuMP model
model = Model(Ipopt.Optimizer)

# Number of points defining a path
num_points = 50

# Decision variables for the x and y coordinates
@variable(model, x[1:num_points])
@variable(model, y[1:num_points])

# A path consists of straightline segments from 
# (x[i], y[i]) to (x[i+1], y[i+1]), i=1:num_points-1
# hence, a piecewise linear function

# Initialize start and end points
@constraint(model, x[1] == start_point[1])
@constraint(model, y[1] == start_point[2])
@constraint(model, x[num_points] == end_point[1])
@constraint(model, y[num_points] == end_point[2])

# Objective: Minimize the total path length
@NLobjective(model, Min, sum(((x[i+1] - x[i])^2 + (y[i+1] - y[i])^2 + 1e-8) for i in 1:num_points-1))

# Use a circle as an obstacle avoidance constraint for each point in the path
for i in 1:num_points
    @NLconstraint(model, (x[i] - circle_center[1])^2 + (y[i] - circle_center[2])^2 >= circle_radius^2)
end

set_optimizer_attribute(model, "print_level", 0)

# Solve the optimization problem
optimize!(model)

# Extract the optimized path
optimized_x = value.(x)
optimized_y = value.(y)



In [ ]:

# Plotting
p = plot(aspect_ratio=:equal, legend=:right)

# Plot optimized path
plot!(optimized_x, optimized_y, label="Optimized Path", line=:solid, marker=:circle)

# Plot circle obstacle
circle_theta = 0:0.01:2*π
plot!(circle_center[1] .+ circle_radius .* cos.(circle_theta), circle_center[2] .+ circle_radius .* sin.(circle_theta), 
    label="Circle Obstacle", fill=(0, :orange))

display(p)
### BEGIN HIDDEN TESTS

### END HIDDEN TESTS

### Next, Observe the Iterative Nature of the Convergence Process in Action

In [ ]:
using JuMP, Ipopt, Plots

# Define the start and end points, obstacle characteristics
start_point = [0.0, 0.0]
end_point = [9.0, 10.0]
circle_center = [4.0, 4.0]
circle_radius = 3.0

# Number of points defining the path
num_points = 50

model = Model(Ipopt.Optimizer)

# Decision variables for the x and y coordinates
@variable(model, x[1:num_points])
@variable(model, y[1:num_points])

# Initialize start and end points
@constraint(model, x[1] == start_point[1])
@constraint(model, y[1] == start_point[2])
@constraint(model, x[num_points] == end_point[1])
@constraint(model, y[num_points] == end_point[2])

# Objective: Minimize the total path length
@objective(model, Min, sum(((x[i+1] - x[i])^2 + (y[i+1] - y[i])^2 + 1e-8) for i in 1:num_points-1))


# Circle obstacle avoidance constraints for each point
for i in 1:num_points
    @constraint(model, (x[i] - circle_center[1])^2 + (y[i] - circle_center[2])^2 >= circle_radius^2)
end

set_optimizer_attribute(model, "print_level", 0)

# Initialize plot
plt = plot(aspect_ratio=:equal, legend=:topright)

# Plot the circle obstacle
circle_theta = 0:0.01:2π
plot!(plt, circle_center[1] .+ circle_radius .* cos.(circle_theta), 
         circle_center[2] .+ circle_radius .* sin.(circle_theta), 
         label="Circle Obstacle", fill=(0, :orange))

optimized_x =[]; optimized_y=[]

# # Loop to simulate the iterative optimization process
# for iteration in 1:15
    
#     # Limit Ipopt to a fixed number of iterations for each "learning" iteration
#     set_optimizer_attribute(model, "max_iter", iteration *15)
    
#     # Solve the model
#     optimize!(model)
    
#     # Extract and plot the optimized path
#     optimized_x = value.(x)
#     optimized_y = value.(y)
#     plot!(plt, optimized_x, optimized_y, label="Iteration $(iteration*15)")
    
#     display(plt)
# end

anim = @animate for i = 1:100
    set_optimizer_attribute(model, "max_iter", i)
    optimize!(model)
    optimized_x = value.(x)
    optimized_y = value.(y)
    plot(plt, optimized_x, optimized_y, legend=false, 
        title="Optimizing the path from $(start_point) to $(end_point)\nIteration $(i)")
end

# Run the next cell to see the gif

In [ ]:
plot(plt, optimized_x, optimized_y, legend=false, 
        title="Optimizing the path from $(start_point) to $(end_point)\nIteration $(1)")

mp4(anim, "pathplanning_optim_single_Obstacle.mp4", fps=10)

## Problem 1: Final Feat of Magic: Path Planning with Multiple Obstacles. 

You can play around with the start point, end point, and obstacle locations to create different paths!

<table>
  <tr>
    <td><img src="data/HW07ConvergnceProcessMultiObstacle.png",  width="60%"></td>
    <td><img src="data/HW07ConvergnceProcessMultiObstacleMiddle.png",  width="60%"></td>
      </tr>
</table>

In [ ]:
using JuMP, Ipopt, Plots

# Define the start and end points, obstacle characteristics
start_point = [0.0, 0.0]
end_point = [31.0, 32.0]

circle1_center = [4.0, 4.0]
circle1_radius = 2.0

circle2_center = [10, 10.5]
circle2_radius = 1.25

circle3_center = [18.5, 18]
circle3_radius = 3

circle4_center = [24.5, 24.5]
circle4_radius = 2.15

circle5_center = [30.0, 30.5]
circle5_radius = 0.5


# Number of points defining the path
num_points = 50

model = Model(Ipopt.Optimizer)

# Decision variables for the x and y coordinates
@variable(model, x[1:num_points])
@variable(model, y[1:num_points])

# Initialize start and end points
@constraint(model, x[1] == start_point[1])
@constraint(model, y[1] == start_point[2])
@constraint(model, x[num_points] == end_point[1])
@constraint(model, y[num_points] == end_point[2])

# Objective: Minimize the total path length
# @objective(model, Min, ...)
### BEGIN SOLUTION
@objective(model, Min, sum(((x[i+1] - x[i])^2 + (y[i+1] - y[i])^2 + 1e-8) for i in 1:num_points-1))
### END SOLUTION

# Circle obstacle avoidance constraints for each point
extra_margin_for_radius = 1e-4
for i in 1:num_points
    # @constraint(model, ...)
    ### BEGIN SOLUTION
    @constraint(model, (x[i] - circle1_center[1])^2 + (y[i] - circle1_center[2])^2 >= (circle1_radius + extra_margin_for_radius)^2)
    @constraint(model, (x[i] - circle2_center[1])^2 + (y[i] - circle2_center[2])^2 >= (circle2_radius + extra_margin_for_radius)^2)
    @constraint(model, (x[i] - circle3_center[1])^2 + (y[i] - circle3_center[2])^2 >= (circle3_radius + extra_margin_for_radius)^2)
    @constraint(model, (x[i] - circle4_center[1])^2 + (y[i] - circle4_center[2])^2 >= (circle4_radius + extra_margin_for_radius)^2)
    @constraint(model, (x[i] - circle5_center[1])^2 + (y[i] - circle5_center[2])^2 >= (circle5_radius + extra_margin_for_radius)^2)
    ### END SOLUTION
end

set_optimizer_attribute(model, "print_level", 0)

# Initialize plot
plt = plot(aspect_ratio=:equal, xlims=(0,35), ylims=(0,35))

# Plot the circle obstacles
circle_theta = 0:0.01:2π
plot!(plt, circle1_center[1] .+ circle1_radius .* cos.(circle_theta), 
         circle1_center[2] .+ circle1_radius .* sin.(circle_theta), 
         fill=(0, :orange))

plot!(plt, circle2_center[1] .+ circle2_radius .* cos.(circle_theta), 
         circle2_center[2] .+ circle2_radius .* sin.(circle_theta), 
         fill=(0, :orange))

plot!(plt, circle3_center[1] .+ circle3_radius .* cos.(circle_theta), 
         circle3_center[2] .+ circle3_radius .* sin.(circle_theta), 
         fill=(0, :orange))

plot!(plt, circle4_center[1] .+ circle4_radius .* cos.(circle_theta), 
         circle4_center[2] .+ circle4_radius .* sin.(circle_theta), 
         fill=(0, :orange))

plot!(plt, circle5_center[1] .+ circle5_radius .* cos.(circle_theta), 
        circle5_center[2] .+ circle5_radius .* sin.(circle_theta), 
        fill=(0, :orange))

optimized_x =[]; optimized_y=[]


anim = @animate for i = 1:2:200
    set_optimizer_attribute(model, "max_iter", i)
    optimize!(model)
    optimized_x = value.(x)
    optimized_y = value.(y)
    plot(plt, optimized_x, optimized_y, legend=false, 
        title="Optimizing the path from $(start_point) to $(end_point)\nIteration $(i)")
end

mp4(anim, "pathplanning_optim.mp4", fps=10)


In [ ]:
#= Friendly Check =#
using LinearAlgebra
set_optimizer_attribute(model, "max_iter", 200)
optimize!(model)
optimized_x = value.(x)
optimized_y = value.(y)

optimized_x_ans_first_5 = [-1.9158205754706475e-31, 0.8381762502895878, 1.6763525012832179, 2.514528753065119, 3.352705005741577]
optimized_x_ans_last_5 = [28.011279770154648, 28.75845982060225, 29.505639873023235, 30.25281993718988, 31.0]
optimized_y_ans_first_5 = [4.1468949145610476e-31, 0.38082286193215453, 0.7616457246431918, 1.1424685883632593, 1.523291453567372]
optimized_y_ans_last_5 = [29.767178947285714, 30.32538421661505, 30.88358948577485, 31.441794744797914, 32.0]

is_it_correct_checkx1 = norm(optimized_x[1:5] - optimized_x_ans_first_5) <= 1e-2 ? "Yes" : "No"
is_it_correct_checkx2 = norm(optimized_x[46:50] - optimized_x_ans_last_5) <= 1e-2 ? "Yes" : "No"
is_it_correct_checky1 = norm(optimized_y[1:5] - optimized_y_ans_first_5) <= 1e-2 ? "Yes" : "No"
is_it_correct_checky2 = norm(optimized_y[46:50] - optimized_y_ans_last_5) <= 1e-2 ? "Yes" : "No"

@show is_it_correct_checkx1;
@show is_it_correct_checkx2;
@show is_it_correct_checky1;
@show is_it_correct_checky2;

# Computing Antiderivatives with SymPy

<p align="center">
  <img src="data/HW07AntiderivativeDefinition.png", width="90%">
</p>
<br>

Review the following cell carefully, then use what you’ve learned to solve the problems that follow.


In [ ]:
using SymPy

@syms x

# Complicated Rational Function
f = (x^6 - 2x^4 + 3x^2 - 4) / (x^3 - x)

# To display the functions:
println("Function f: ", f)

# Compute the antiderivative
F = integrate(f, x)

# Display the result
println("\nThe antiderivative of ", f, " is \n", F, "\n")

# Proof that F1 is an antiderivative of f1
@show Check = simplify(f - diff(F, x))

# Problem 2: Compute Velocity and Displacement from Acceleration

Given the acceleration function $A(x) = sin(x^2) * cos(x)$, find the velocity and displacement function. (Use SymPy)

In [ ]:
using SymPy

@syms x

A = sin(x^2) * cos(x)

# Uncomment and fill in the answers.
# V =
# X =

### BEGIN SOLUTION 
V = integrate(A, x)
X = integrate(V, x)
### END SOLUTION

In [ ]:
# friendly check
# if the value of is_it_correct_checkN is "Yes", then your answer is LIKELY correct. 
# If the value of is_it_correct_checkN is "No", then your answer is DEFINITELY wrong

is_it_correct_checkV = simplify(A - diff(V, x)) == 0 ? "Yes" : "No"
is_it_correct_checkX = simplify(V - diff(X, x)) == 0 ? "Yes" : "No"

@show is_it_correct_checkV;
@show is_it_correct_checkX;

# Fundamental Theorems of Calculus

<p align="center">
  <img src="data/HW07FundamentalTheoremsCalculus.png", width="90%">
</p>
<br>

  - **The First Fundamental Theorem** states that "*if you integrate a function and then differentiate it (with respect to the upper limit of integration), you get the original function back*". That is, differentiation undoes (inverses) integration, just as first exponentiating a number and then taking the logarithm gives back the original number. The logarithm undoes (inverses) exponentiation.
  
  
  - **The Second Fundamental Theorem** states that "*if you differentiate a function and then integrate it, you recover the original function up to an arbitrary constant, typically denoted $C$.*" The appearance of the arbitray constant is because $f(x)$ and $f(x) + C$ have the same derivative. Hence, modulo this constant of integration, integration undoes (inverses) differentiation. An analogy in the real numbers would be, if you compute the sine function of an arbitray real number and then apply arcsine to it, you will recover the original number modulo $\pi$. This is because the range of the arcsine function is $[-\frac{\pi}{2},  \frac{\pi}{2}]$.


In [ ]:
# Illustration of the First Fundamental Theorem of Calculus

using ForwardDiff

# Define the function
f(x) = exp(x^2)

# Simple trapezoidal rule for integration compatible with ForwardDiff
function trapZGPTstyle(f, a, b, n=1000)
    h = (b - a) / n
    if true # suggested by GPT
        integral = 0.5 * (f(a) + f(b))
        for i in 1:n-1
            x = a + i * h
            integral += f(x)
        end
        return integral * h
        else # our standard formulation which is less efficient
        integral = 0.0
        xi = a
         for i in 1:n
            xip1 = a + i*h
            integral += h*(f(xi) + f(xip1))/2.0
            xi = xip1
        end
        return integral        
    end
end

# Define the antiderivative using the trapezoidal rule
function antiderivative(x)
    return trapZGPTstyle(f, 0, x)
end

# Define a function to differentiate the antiderivative
g(x) = antiderivative(x)

# Differentiate the antiderivative at a specific point using ForwardDiff
x0 = 1.0  # Point at which to differentiate
df_numerical = ForwardDiff.derivative(g, x0)

# Compare with the analytical derivative
df_analytical = f(x0)

println("Numerical derivative at x = $x0: $df_numerical")
println("Analytical value of f(x) at x = $x0: $df_analytical")
println("Error: ", abs(df_numerical - df_analytical))

println("\nThe derivative of an indefinite integral is the INTEGRAND (the function 
being integrated). This is the message of the First Fundamental Theorem of Calculus.")

In [ ]:
# Illustration of the Second Fundamental Theorem of Calculus
using Plots, LaTeXStrings
using ForwardDiff
using QuadGK

# Define the original function
#F(x) = x^2 + 3x + 2
F(x) = exp(sin(sqrt(x))) - 2.0

# Derivative function using ForwardDiff
dF(x) = ForwardDiff.derivative(F, x)

# Function to perform the numerical integration of the derivative
function integrated_derivative(x)
    integral, _ = quadgk(dF, 0, x)  # Integrating from 0 to x
    return integral
end

# Generate data for plotting
x_values = 1e-3:0.1:10
original_values = F.(x_values)
integrated_values = integrated_derivative.(x_values)

# Calculate the constant difference. Can you any common point
constant = original_values[11] - integrated_values[11]

# Plotting the functions
p1 = plot(x_values, original_values, label="Original Function F(x)", linewidth=5, color=:lightgray, guidefont=15)
plot!(x_values, integrated_values, label="Integral of F'(x)", linestyle=:dashdot, linewidth=2, color=:blue)
plot!(x_values, integrated_values .+ constant, label="Integral of F'(x) + constant", linestyle=:dash, 
    color=:red, linewidth=2)
title!("Second Fundamental Theorem of Calculus in Action")
xlabel!(L"x")
ylabel!(L"F(x)")

println("\nThe integral of the derivative of a function recovers the original function
up to a constant. This is the message of the Second Fundamental Theorem of Calculus.\n \n")


display(p1)

# LLMs and Calculus

<p align="center">
  <img src="data/HW07ChatGPTSelfPortrait.png", width="40%">
</p>
<br>

The following questions are based on the Fundamental Theorems of Calculus. Open this cell by clicking on it, and then copy-paste them into your favorite LLM along with a request for detailed answers. Based on the questions and answers, respond to the questions in the next cell.

1. Given $f(x) = 5x^4 - 3x + 2$ and define $G(x) = \int_{0}^{x} f(t) \, dt$. Use the First Fundamental Theorem of Calculus to find the derivative of $G(x)$.

2. Let $\beta(x) = \cos(x^2) \cdot \ln(1+x)$ and it is known that $\beta'(x) = \frac{-2x \sin(x^2) \cdot \ln(1+x) + \cos(x^2)}{1+x}$. Compute $\int_{0}^{2} \beta'(x) \, dx$.

3. Suppose $p(x)$ is a continuous function such that $\int_{-1}^{x} p(t) \, dt = 3x^3 + x^2 - x$. Determine $p(x)$.

4. Given that $r(t)$ satisfies $\int_{3}^{t} r(x) \, dx = 5t - t^2$, calculate $r(t)$.


# Problem 3: Fundamental Theorems of Calculus and LLMs

Here are four multiple-choice questions relating to the Fundamental Theorem of Calculus, with the last one involving the use of large language models (LLMs) to generate solutions:

**Question 3.1a: Let $G(x) = \int_{1}^{x} \cos(t^2) dt$. Which of the following is the correct expression for $G'(x)$?**
   - A) $\cos(x^2)$
   - B) $\cos(x^2) \cdot 2x$
   - C) $\sin(x^2)$
   - D) $\sin(x^2) \cdot 2x$

**Question 3.1b: Which of the following has different value?**
   - A) $\frac{d}{dt} (\int_{1}^{x} \cos(t^2) dt)$
   - B) $\frac{d}{dt} (\int_{1}^{x} \cos(k^2) dk)$
   - C) $\frac{d}{dt} (\int_{3}^{x} \cos(t^2) dt)$
   - D) All the above are the same.

**Question 3.2: If $\int_{x}^{c} j(t) \, dt = x^5 + 3x^2 + 7$, then $j(x)$ equals:**
   - A) $5x^4 + 6x$
   - B) $x^5 + 3x^2 + 7$
   - C) $- 5x^4 - 6x$
   - D) $x^5 + 3x^2$


**Question 3.3: When using a large language model (LLM) like GPT-4 to support learning calculus, particularly with the Fundamental Theorem of Calculus, which of the following best reflects proper use?**
   - A) LLMs can generate exact, step-by-step proofs for all integrals without error.
   - B) LLMs replace the need to study since they provide answers instantly.
   - C) LLMs are useful tools for exploring reasoning, checking steps, and clarifying methods when used alongside human understanding.
   - D) LLMs generate answers using a fixed rulebook of human-written solutions and cannot handle new problems.

### Instructions
Record your answers in the cell below using uppercase (CAPITAL) letters: A, B, C, or D.



In [ ]:
# Answer1 = "X"
# Answer2 = "X"
# Answer3 = "X"
# Answer4 = "X"

### BEGIN SOLUTION 
Answer1 = "A"
Answer2 = "D"
Answer3 = "C"
Answer4 = "C"
### END SOLUTION 


In [ ]:
# Friendly Check
function evaluate_answers(answer_code)
    # This is an obfuscating mechanism using a pseudo-hash function with an additional obfuscation key
    function pseudo_hash(c, i, obfuscation_key)
        ascii_val = Int(c)  # Convert character to its ASCII integer value
        # Apply a more complex hashing function that uses both the position and a key
        return (ascii_val % 16 + i * (ascii_val % 3) + obfuscation_key[i] % 7) % 11
    end
    
    # Define a constant obfuscation key (could be randomly generated for each session or exam)
    obfuscation_key = [3, 6, 5, 7]

    # Compute the pseudo-hash for each student answer based on its position and the obfuscation key
    student_hashes = [pseudo_hash(Char(ans), idx, obfuscation_key) for (idx, ans) in enumerate(answer_code)]

    # Compare hashes to determine the number of correct answers
    # Obfuscated problem hashes, precomputed using the same obfuscation key
    problem_hashes = [6, 3, 0, 7] # Example hashes that you would compute beforehand
    num_correct = sum([student_hashes[i] == problem_hashes[i] for i in 1:length(answer_code)])
    
    return num_correct
end

CheckAnswers = Answer1*Answer2*Answer3*Answer4
NumCorrect = evaluate_answers(CheckAnswers)
println("You currently have $NumCorrect correct answers out of 4. 
    They are worth 0.5 points each.")

<p align="center" style="font-size:48px;"><strong>The End! </strong></p>